# Surface heat fluxes in the Nino3/4/3.4 regions
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

#### Mechanics
Dependencies

In [1]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from matplotlib import rcParams
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.io import loadmat
from matplotlib.patches import Polygon
from scipy import signal
from scipy import stats

Model names

In [2]:
modelnames_fig = ['CCSM3 abrupt 2x','CCSM3 abrupt 4x','CCSM3 abrupt 8x', \
    'CESM1.0.4 abrupt 2x','CESM1.0.4 abrupt 4x','CESM1.0.4 abrupt 8x', 'CNRM-CM6.1 abrupt4x', \
    'GFDL-ESM2M 1pct 2x','GISS-E2-R 1pct 4x', \
    'GISS-E2-R abrupt 4x','HadCM3L abrupt 2x','HadCM3L abrupt 4x', \
    'HadCM3L abrupt 6x','HadCM3L abrupt 8x','IPSL-CM5A-LR abrupt 4x', \
    'MIROC3.2 1pct 2x','MIROC3.2 1pct 4x','MPIESM-1.2 abrupt 2x', \
    'MPIESM-1.2 abrupt 4x','MPIESM-1.2 abrupt 8x']

modelnames_file = ['CCSM3_abrupt2x','CCSM3_abrupt4x','CCSM3_abrupt8x', \
    'CESM104_abrupt2x','CESM104_abrupt4x','CESM104_abrupt8x', \
    'CNRMCM61_abrupt4x','GFDLESM2M_1pct2x','GISSE2R_1pct4x', \
    'GISSE2R_abrupt4x','HadCM3L_abrupt2x','HadCM3L_abrupt4x', \
    'HadCM3L_abrupt6x','HadCM3L_abrupt8x','IPSLCM5A_abrupt4x', \
    'MIROC32_1pct2x','MIROC32_1pct4x','MPIESM12_abrupt2x', \
    'MPIESM12_abrupt4x','MPIESM12_abrupt8x']

Data locations

In [3]:
loc_hfls = "~/" # change if using raw data
loc_hfss = "~/" # change if using raw data
loc_out = "../Data/HeatFlux/"

Custom detrending function

In [4]:
def custom_detrend(data,order):
    
    # only for one-dimensional timeseries
    # numpy required
    
    x = np.arange(1,len(data)+1,1)
    
    model = np.polyfit(x,data,order)
    predicted = np.polyval(model,x)
    new = data - predicted
    
    return(new)

Function for listing files

In [5]:
def list_files(directory, extension):
    
    # Credit to: http://www.martinbroadhurst.com/listing-all-files-in-a-directory-with-a-certain-extension-in-python.html
    # Requires os.listdir
    
    return (f for f in os.listdir(directory) if f.endswith(extension))

Index information

In [6]:
indices = ["nino3","nino34","nino4"]

lat_mins = [-5,-5,-5]
lat_maxs = [5,5,5]
lon_mins = [210,190,160]
lon_maxs = [270,240,210]

Detrending order

In [7]:
ordr = 2

In [8]:
for i in np.arange(0,len(modelnames_file),1):
    
    model, exp = modelnames_file[i].split("_")
    print(modelnames_file[i])
    
    fname_hfss_exp = [f for f in os.listdir(loc_hfss) if f.startswith("hfss_mon_"+modelnames_file[i])][0]
    fname_hfss_control = [f for f in os.listdir(loc_hfss) if f.startswith("hfss_mon_"+model+"_control")][0]
    
    fname_hfls_exp = [f for f in os.listdir(loc_hfls) if f.startswith("hfls_mon_"+modelnames_file[i])][0]
    fname_hfls_control = [f for f in os.listdir(loc_hfls) if f.startswith("hfls_mon_"+model+"_control")][0]
        
    # In this script we're repeatedly doing the calculations for the control, for each experiment
    # That is, this loop for CESM abrupt2x and abrupt4x does the exact same calculation on the
    # CESM control and re-writes the file
    # but I'm too lazy to be more precise about it
    
    model_decode_times_list = ["MIROC32","MPIESM12"]
    
    if model in model_decode_times_list:
        dc = False
    else:
        dc = True
        
    hfss_exp_global = xr.DataArray(xr.open_dataset(loc_hfss+fname_hfss_exp,decode_times=dc).data_vars["hfss"]) 
    hfss_control_global = xr.DataArray(xr.open_dataset(loc_hfss+fname_hfss_control,decode_times=dc).data_vars["hfss"])
    
    hfls_exp_global = xr.DataArray(xr.open_dataset(loc_hfls+fname_hfls_exp,decode_times=dc).data_vars["hfls"]) 
    hfls_control_global = xr.DataArray(xr.open_dataset(loc_hfls+fname_hfls_control,decode_times=dc).data_vars["hfls"])
    
    hfnet_exp_global = hfss_exp_global + hfls_exp_global
    hfnet_control_global = hfss_control_global + hfls_control_global
    
    if model == "HadCM3L":
        lat = hfnet_exp_global.coords["latitude_1"]
        lon = hfnet_exp_global.coords["longitude_1"]
        latname = "latitude_1"
        lonname = "longitude_1"
    else:
        lat = hfnet_exp_global.coords["lat"]
        lon = hfnet_exp_global.coords["lon"]
    
    # loop through indices
    
    for j in np.arange(0,len(indices),1):
        print(indices[j])
        
        if lat[0] > lat[len(lat.values)-1]:
            latmax = lat_mins[j]
            latmin = lat_maxs[j]
        else:
            latmax = lat_maxs[j]
            latmin = lat_mins[j]
            
        # area-average
        hfss_index_exp_raw = hfss_exp_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        hfss_index_control_raw = hfss_control_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        
        hfls_index_exp_raw = hfls_exp_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        hfls_index_control_raw = hfls_control_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        
        hfnet_index_exp_raw = hfnet_exp_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        hfnet_index_control_raw = hfnet_control_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        
        # time coordinates
        n_months_c_hfss = hfss_index_control_raw.shape[0]
        n_months_f_hfss = hfss_index_exp_raw.shape[0]
        time_exp_hfss = xr.cftime_range(start='0001', periods=n_months_f_hfss, freq='M')
        time_control_hfss = xr.cftime_range(start='0001', periods=n_months_c_hfss, freq='M')
        
        n_months_c_hfls = hfls_index_control_raw.shape[0]
        n_months_f_hfls = hfls_index_exp_raw.shape[0]
        time_exp_hfls = xr.cftime_range(start='0001', periods=n_months_f_hfls, freq='M')
        time_control_hfls = xr.cftime_range(start='0001', periods=n_months_c_hfls, freq='M')
        
        n_months_c_hfnet = hfnet_index_control_raw.shape[0]
        n_months_f_hfnet = hfnet_index_exp_raw.shape[0]
        time_exp_hfnet = xr.cftime_range(start='0001', periods=n_months_f_hfnet, freq='M')
        time_control_hfnet = xr.cftime_range(start='0001', periods=n_months_c_hfnet, freq='M')
            
        # attach time coords to calculate anomalies
        hfss_index_exp_time = xr.DataArray(hfss_index_exp_raw.values,coords=[time_exp_hfss],dims=["time"])
        hfss_index_control_time = xr.DataArray(hfss_index_control_raw.values,coords=[time_control_hfss],dims=["time"])
        
        hfls_index_exp_time = xr.DataArray(hfls_index_exp_raw.values,coords=[time_exp_hfls],dims=["time"])
        hfls_index_control_time = xr.DataArray(hfls_index_control_raw.values,coords=[time_control_hfls],dims=["time"])
        
        hfnet_index_exp_time = xr.DataArray(hfnet_index_exp_raw.values,coords=[time_exp_hfnet],dims=["time"])
        hfnet_index_control_time = xr.DataArray(hfnet_index_control_raw.values,coords=[time_control_hfnet],dims=["time"])
        
        
        
        print("detrending and calculating anomalies...")
        # detrend fast/transient/eq (if abrupt) and transient/eq (if 1pct)
        # then calculate anomalies
        
        # detrend control
        hfss_index_ctrl_detrend = xr.DataArray(custom_detrend(hfss_index_control_time,1).values,coords=[time_control_hfss],dims=["time"])
        hfss_index_ctrl_anom = hfss_index_ctrl_detrend.groupby("time.month") - (hfss_index_ctrl_detrend.groupby("time.month").mean(dim="time"))
        
        hfls_index_ctrl_detrend = xr.DataArray(custom_detrend(hfls_index_control_time,1).values,coords=[time_control_hfls],dims=["time"])
        hfls_index_ctrl_anom = hfls_index_ctrl_detrend.groupby("time.month") - (hfls_index_ctrl_detrend.groupby("time.month").mean(dim="time"))

        hfnet_index_ctrl_detrend = xr.DataArray(custom_detrend(hfnet_index_control_time,1).values,coords=[time_control_hfnet],dims=["time"])
        hfnet_index_ctrl_anom = hfnet_index_ctrl_detrend.groupby("time.month") - (hfnet_index_ctrl_detrend.groupby("time.month").mean(dim="time"))
        
        flux_indices = ["hfss","hfls","hfnet"]
        for cc in np.arange(0,len(flux_indices),1):
            flux_index = flux_indices[cc]
            if flux_index == "hfss":
                index_control_raw = hfss_index_control_time
                index_control_out = hfss_index_ctrl_anom
                index_exp = hfss_index_exp_time
            elif flux_index == "hfls":
                index_control_raw = hfls_index_control_time
                index_control_out = hfls_index_ctrl_anom
                index_exp = hfls_index_exp_time
            else:
                index_control_raw = hfnet_index_control_time
                index_control_out = hfnet_index_ctrl_anom
                index_exp = hfnet_index_exp_time
                
            # write out raw index from control and exp
            index_control_raw.name = flux_index
            index_control_raw.attrs["creation_date"] = str(datetime.datetime.now())
            index_control_raw.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            index_control_raw.attrs["data_description"] = flux_index+" in "+indices[j]+" region  from "+model+" control, raw"
            index_control_raw.attrs["created_from"] = "Calculate_HeatFlux_Indices.ipynb"

            fname_out_control_anom = loc_out+flux_index+"_"+indices[j]+"_"+model+"_control.nc"
            index_control_raw.to_netcdf(fname_out_control_anom,mode="w")
            print(fname_out_control_anom)
        
            # write out index from experiment
            index_exp.name = flux_index
            index_exp.attrs["creation_date"] = str(datetime.datetime.now())
            index_exp.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            index_exp.attrs["data_description"] = flux_index+" in "+indices[j]+" region from "+model+" "+exp+", raw"
            index_exp.attrs["created_from"] = "Calculate_HeatFlux_Indices.ipynb"

            fname_out_exp = loc_out+flux_index+"_"+indices[j]+"_"+model+"_"+exp+".nc"
            index_exp.to_netcdf(fname_out_exp,mode="w")
            print(fname_out_exp)
        
            
            
            # write out anomaly index from control 
            index_control_out.name = flux_index
            index_control_out.attrs["creation_date"] = str(datetime.datetime.now())
            index_control_out.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            index_control_out.attrs["data_description"] = flux_index+" in "+indices[j]+" region  from "+model+" control, anomaly and linearly detrended"
            index_control_out.attrs["created_from"] = "Calculate_HeatFlux_Indices.ipynb"

            fname_out_control_anom = loc_out+flux_index+"_"+indices[j]+"_"+model+"_control_anom_detrend.nc"
            index_control_out.to_netcdf(fname_out_control_anom,mode="w")
            print(fname_out_control_anom)
        
            # conditional for abrupt/1pct
            ntimes_peryear = 12

            if exp.startswith("abrupt"):

                index_fast = index_exp[0:(50*ntimes_peryear)]
                index_transient = index_exp[(50*ntimes_peryear):(150*ntimes_peryear)]
                index_eq = index_exp[(150*ntimes_peryear):]

                index_fast_detrend = xr.DataArray(custom_detrend(index_fast,ordr).values,coords=[index_fast.coords["time"]],dims=["time"])
                index_transient_detrend = xr.DataArray(custom_detrend(index_transient,ordr).values,coords=[index_transient.coords["time"]],dims=["time"])
                index_eq_detrend = xr.DataArray(custom_detrend(index_eq,ordr).values,coords=[index_eq.coords["time"]],dims=["time"])

                index_fast_anom = index_fast_detrend.groupby("time.month") - (index_fast_detrend.groupby("time.month").mean(dim="time"))
                index_transient_anom = index_transient_detrend.groupby("time.month") - (index_transient_detrend.groupby("time.month").mean(dim="time"))
                index_eq_anom = index_eq_detrend.groupby("time.month") - (index_eq_detrend.groupby("time.month").mean(dim="time"))

                index_exp_anom = xr.concat([index_fast_anom,index_transient_anom,index_eq_anom],dim="time")

            else:

                index_transient = index_exp[0:(140*ntimes_peryear)]
                index_eq = index_exp[(140*ntimes_peryear):]

                index_transient_detrend = xr.DataArray(custom_detrend(index_transient,ordr).values,coords=[index_transient.coords["time"]],dims=["time"])
                index_eq_detrend = xr.DataArray(custom_detrend(index_eq,ordr).values,coords=[index_eq.coords["time"]],dims=["time"])

                index_transient_anom = index_transient_detrend.groupby("time.month") - (index_transient_detrend.groupby("time.month").mean(dim="time"))
                index_eq_anom = index_eq_detrend.groupby("time.month") - (index_eq_detrend.groupby("time.month").mean(dim="time"))

                index_exp_anom = xr.concat([index_transient_anom,index_eq_anom],dim="time")


            # write out index from experiment
            index_exp_anom.name = flux_index
            index_exp_anom.attrs["creation_date"] = str(datetime.datetime.now())
            index_exp_anom.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            index_exp_anom.attrs["data_description"] = flux_index+" in "+indices[j]+" region from "+model+" "+exp+", anomaly and detrended using order in file name"
            index_exp_anom.attrs["created_from"] = "Calculate_HeatFlux_Indices.ipynb"

            fname_out_exp_anom = loc_out+flux_index+"_"+indices[j]+"_"+model+"_"+exp+"_anom_detrend"+str(ordr)+".nc"
            index_exp_anom.to_netcdf(fname_out_exp_anom,mode="w")
            print(fname_out_exp_anom)

CCSM3_abrupt2x
nino3
detrending and calculating anomalies...
../Data/HeatFlux/hfss_nino3_CCSM3_control.nc
../Data/HeatFlux/hfss_nino3_CCSM3_abrupt2x.nc
../Data/HeatFlux/hfss_nino3_CCSM3_control_anom_detrend.nc
../Data/HeatFlux/hfss_nino3_CCSM3_abrupt2x_anom_detrend2.nc
../Data/HeatFlux/hfls_nino3_CCSM3_control.nc
../Data/HeatFlux/hfls_nino3_CCSM3_abrupt2x.nc
../Data/HeatFlux/hfls_nino3_CCSM3_control_anom_detrend.nc
../Data/HeatFlux/hfls_nino3_CCSM3_abrupt2x_anom_detrend2.nc
../Data/HeatFlux/hfnet_nino3_CCSM3_control.nc
../Data/HeatFlux/hfnet_nino3_CCSM3_abrupt2x.nc
../Data/HeatFlux/hfnet_nino3_CCSM3_control_anom_detrend.nc
../Data/HeatFlux/hfnet_nino3_CCSM3_abrupt2x_anom_detrend2.nc
nino34
detrending and calculating anomalies...
../Data/HeatFlux/hfss_nino34_CCSM3_control.nc
../Data/HeatFlux/hfss_nino34_CCSM3_abrupt2x.nc
../Data/HeatFlux/hfss_nino34_CCSM3_control_anom_detrend.nc
../Data/HeatFlux/hfss_nino34_CCSM3_abrupt2x_anom_detrend2.nc
../Data/HeatFlux/hfls_nino34_CCSM3_control.nc
..

/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/dartfs-hpc/rc/home/y/f003k8y/.conda/envs/ccallahan/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, or

nino3
detrending and calculating anomalies...
../Data/HeatFlux/hfss_nino3_CNRMCM61_control.nc
../Data/HeatFlux/hfss_nino3_CNRMCM61_abrupt4x.nc
../Data/HeatFlux/hfss_nino3_CNRMCM61_control_anom_detrend.nc
../Data/HeatFlux/hfss_nino3_CNRMCM61_abrupt4x_anom_detrend2.nc
../Data/HeatFlux/hfls_nino3_CNRMCM61_control.nc
../Data/HeatFlux/hfls_nino3_CNRMCM61_abrupt4x.nc
../Data/HeatFlux/hfls_nino3_CNRMCM61_control_anom_detrend.nc
../Data/HeatFlux/hfls_nino3_CNRMCM61_abrupt4x_anom_detrend2.nc
../Data/HeatFlux/hfnet_nino3_CNRMCM61_control.nc
../Data/HeatFlux/hfnet_nino3_CNRMCM61_abrupt4x.nc
../Data/HeatFlux/hfnet_nino3_CNRMCM61_control_anom_detrend.nc
../Data/HeatFlux/hfnet_nino3_CNRMCM61_abrupt4x_anom_detrend2.nc
nino34
detrending and calculating anomalies...
../Data/HeatFlux/hfss_nino34_CNRMCM61_control.nc
../Data/HeatFlux/hfss_nino34_CNRMCM61_abrupt4x.nc
../Data/HeatFlux/hfss_nino34_CNRMCM61_control_anom_detrend.nc
../Data/HeatFlux/hfss_nino34_CNRMCM61_abrupt4x_anom_detrend2.nc
../Data/HeatFlu